# Day 22
## Part 1

In [1]:
steps = []

# with open("ex1.txt") as f:
# with open("ex2.txt") as f:
with open("reactor.txt") as f:
    for line in f.readlines():
        stripped = line.strip()
        command  = stripped.split(" ")[0]
        x_range = stripped.split("x=")[1].split(",")[0]
        y_range = stripped.split("y=")[1].split(",")[0]
        z_range = stripped.split("z=")[1]
        xs = tuple(int(i) for i in x_range.split(".."))
        ys = tuple(int(i) for i in y_range.split(".."))
        zs = tuple(int(i) for i in z_range.split(".."))
        steps.append(tuple((command, xs, ys, zs)))
        
# print(steps)

In [2]:
on = set()
off = set()
for step in steps:
    for x in range(max(-50, step[1][0]), min(50, step[1][1]) + 1):
        for y in range(max(-50, step[2][0]), min(50, step[2][1]) + 1):
            for z in range(max(-50, step[3][0]), min(50, step[3][1]) + 1):
                coord = tuple((x, y, z))
                if step[0] == "on":
                    on.add(coord)
                    if coord in off:
                        off.remove(coord)
                else:
                    off.add(coord)
                    if coord in on:
                        on.remove(coord)
                        
print(len(on))

527915


## Part 2

I need to think of a way to handle this without noting down the state of every single cell. :/

They're all cuboids, so maybe I could just make a list of all cuboids that are on and then deal with splitting them up or something when i have to turn some of them off? I'll need to visualise how a cuboid would split though - into 3 others? 4 others? 6 others because a cuboid has 6 faces? Not sure.

Then at the end I could just calculate volumes for all of the cuboids that are on. :O

There are a few different cases to deal with - a cuboid being entirely encased by another cuboid, a cuboid encasing another cuboid, and a cuboid crossing over with another cuboid along some faces (up to 5 others? I think). I could probably do some logic that works for all of these but I think I'd rather just work on splitting cuboids up when necessary for every case.

The format for a cuboid could just be coordinates of 2 corners. That probably makes more sense than specifying edges or something.

In [27]:
steps = []

with open("ex1.txt") as f:
# with open("ex2.txt") as f:
# with open("reactor.txt") as f:
    for line in f.readlines():
        stripped = line.strip()
        command  = stripped.split(" ")[0]
        x_range = stripped.split("x=")[1].split(",")[0]
        y_range = stripped.split("y=")[1].split(",")[0]
        z_range = stripped.split("z=")[1]
        xs = tuple(int(i) for i in x_range.split(".."))
        ys = tuple(int(i) for i in y_range.split(".."))
        zs = tuple(int(i) for i in z_range.split(".."))
        steps.append(tuple((command, xs, ys, zs)))
        
# print(steps)

What should I do when 2 *on* cuboids intersect? I could either leave them both and deal with splitting them both up in the same way later when an *off* cuboid is found, or I could just split them so that no region is covered by two *on* cuboids later. I think option 2 honestly sounds better, since I need to make a function or something that splits cuboids up anyway to simplify it.

In [12]:
test = steps[0]
print(test)

top_left = (min(test[1]), min(test[2]), min(test[3]))
bottom_right = (max(test[1]), max(test[2]), max(test[3]))
print(top_left, bottom_right)  # the two corners

('on', (10, 12), (10, 12), (10, 12))
(10, 10, 10) (12, 12, 12)


In [34]:
# print(steps[0])
# print(len(steps))

for i in steps:
    for j in i[1:]:
        # print(j)
        assert j[0] <= j[1]

Split any cuboid into 6 sections:
1. directly above the intersecting one (same horizontal area)
2. directly below the intersecting one (same horizontal area)
3. to the left of the intersecting one (same vertical area as the larger)
4. to the right of the intersecting one (same vertical area as the larger)
5. in front of the intersecting one (same vertical area as the larger)
6. behind of the intersecting one (same vertical area as the larger)

In [41]:
# first, cuboids are only overlapping if a corner of cuboid 2 is in the range of cuboid 1
# or if cuboid 2 completely surrounds cuboid 1
def check_overlap(cuboid1, cuboid2):
    """Returns True / False depending on if any further checks need to be done for cuboids.
    The cuboids are made out of 2 corners (x, y, z)."""
    for corner in cuboid2:
        if cuboid1[0][0] <= corner[0] <= cuboid1[1][0]\
        and cuboid1[0][1] <= corner[1] <= cuboid1[1][1]\
        and cuboid1[0][2] <= corner[2] <= cuboid1[1][2]:
            return True
    # check if cuboid 2 surrounds cuboid 1
    if all(cuboid2[0][i] <= cuboid1[0][i] for i in range(0, 3))\
    and all(cuboid2[1][i] >= cuboid1[1][i] for i in range(0, 3)):
        return True
    # check if cuboid 1 surrounds cuboid 2
    if all(cuboid1[0][i] <= cuboid2[0][i] for i in range(0, 3))\
    and all(cuboid1[1][i] >= cuboid2[1][i] for i in range(0, 3)):
        return True
    # are there any other cases? probably lol :(
    return False
    

In [45]:
test1 = ((0, 0, 0), (1, 1, 1))
test2 = ((0.25, 0.25, 0.25), (0.75, 0.75, 0.75))
assert check_overlap(test1, test2)  # test1 surrounds test2

test2 = ((0, 0, 0), (0.75, 0.75, 0.75))
assert check_overlap(test1, test2)

# test2 surrounds test1
test2 = ((-1, -1, -1), (2, 2, 2))
assert check_overlap(test1, test2)

assert check_overlap(test1, test1)  # same shape cuboid overlaps with itself

test2 = ((-1, -1, -1), (0.75, 0.75, 0.75))
assert check_overlap(test1, test2)

test2 = ((-3, -3, -3), (-1, -1, -1))
assert not check_overlap(test1, test2)

test2 = ((-5, -7.5, -2), (0, 0, -1))
assert not check_overlap(test1, test2)

In [ ]:
# make a function to return the volume that is overlapped between 2 cuboids
# What returns if one is surrounded by the other? :/
# if cuboid 1 surrounds cuboid 2, returns cuboid 2?
# if cuboid 2 surrounds cuboid 1, returns cuboid 1?
def get_overlap_section(cuboid1, cuboid2):
    "((x1, y1, z1), (x2, y2, z2)) for each cuboid."
    # check if cuboid 2 surrounds cuboid 1
    if all(cuboid2[0][i] <= cuboid1[0][i] for i in range(0, 3))\
    and all(cuboid2[1][i] >= cuboid1[1][i] for i in range(0, 3)):
        return cuboid1
    # check if cuboid 1 surrounds cuboid 2
    if all(cuboid1[0][i] <= cuboid2[0][i] for i in range(0, 3))\
    and all(cuboid1[1][i] >= cuboid2[1][i] for i in range(0, 3)):
        return cuboid2
    
    # what is the shared volume?

Ok, I'm confusing myself now. I'm going to make a Cuboid class and see if that simplifies things a bit.
REMEMBER TO ADD 0.5 TO POSITIONS DUE TO HOW FLOATING POINTS VS JUST CELLS WORK

In [89]:
class Cuboid:
    def __init__(self, x1, y1, z1, x2, y2, z2, state="on"):
        # x1, y1, z1 are smaller than x2, y2, z2
        self.left = x1
        self.right = x2
        self.bottom = y1
        self.top = y2
        self.front = z1
        self.back = z2
        self.state = state
        
    def __str__(self):
        return f"Cuboid (x1, y1, z1) = ({self.left}, {self.bottom}, {self.front}), (x2, y2, z2) = ({self.right}, {self.top}, {self.back})"
        
    def get_width(self):
        return abs(self.right - self.left)
    
    def get_height(self):
        return abs(self.top - self.bottom)
    
    def get_depth(self):
        return abs(self.back - self.front)
    
    def get_volume(self):
        return self.get_width() * self.get_height() * self.get_depth()
    
    def exists(self):
        "Cuboid exists if it has a width, height, and depth greater than 0."
        return all((self.get_width() > 0, self.get_height() > 0, self.get_depth() > 0))
    
    def is_intersect(self, cuboid):
        # Check if all the way to the left, right, up, down, front, or back compared to other
        if self.left >= cuboid.right or self.right <= cuboid.left:
            return False
        if self.bottom >= cuboid.top or self.top <= cuboid.bottom:
            return False
        if self.front >= cuboid.back or self.back <= cuboid.front:
            return False
        return True
    
    def find_intersect(self, cuboid):
        # Creates a new cuboid that is the intersection between self and input cuboid
        intersect_left = max(self.left, cuboid.left)
        intersect_right = min(self.right, cuboid.right)
        intersect_bottom = max(self.bottom, cuboid.bottom)
        intersect_top = min(self.top, cuboid.top)
        intersect_front = max(self.front, cuboid.front)
        intersect_back = min(self.back, cuboid.back)
        intersect_cuboid = Cuboid(intersect_left, intersect_bottom, intersect_front, 
                                  intersect_right, intersect_top, intersect_back)
        return intersect_cuboid
    
    def split_around_intersect(self, cuboid):
        """Returns a list of cuboids.
        These cuboids are equal to this cuboid minus the other cuboid.
        At most, 6 (six!!!) cuboids should be returned.
        These 6 are: above, below, left, right, in front, behind"""
        
        if not self.is_intersect(cuboid):
            raise ValueError("Cuboid and this cuboid do not intersect!")
            
        above_left = cuboid.left
        above_right = cuboid.right
        above_bottom = cuboid.top
        above_top = self.top
        above_front = cuboid.front
        above_back = cuboid.back
        above_cuboid = Cuboid(above_left, above_bottom, above_front,
                              above_right, above_top, above_back)
        
        below_left = cuboid.left
        below_right = cuboid.right
        below_bottom = self.bottom
        below_top = cuboid.bottom
        below_front = cuboid.front
        below_back = cuboid.back
        below_cuboid = Cuboid(below_left, below_bottom, below_front,
                              below_right, below_top, below_back)
        
        left_left = self.left
        left_right = cuboid.left
        left_bottom = self.bottom
        left_top = self.top
        left_front = cuboid.front
        left_back = cuboid.back
        left_cuboid = Cuboid(left_left, left_bottom, left_front,
                             left_right, left_top, left_back)
        
        right_left = cuboid.right
        right_right = self.right
        right_bottom = self.bottom
        right_top = self.top
        right_front = cuboid.front
        right_back = cuboid.back
        right_cuboid = Cuboid(right_left, right_bottom, right_front,
                              right_right, right_top, right_back)
        
        infront_left = self.left
        infront_right = self.right
        infront_bottom = self.bottom
        infront_top = self.top
        infront_front = self.front
        infront_back = cuboid.front
        infront_cuboid = Cuboid(infront_left, infront_bottom, infront_front,
                                infront_right, infront_top, infront_back)
        
        behind_left = self.left
        behind_right = self.right
        behind_bottom = self.bottom
        behind_top = self.top
        behind_front = cuboid.back
        behind_back = self.back
        behind_cuboid = Cuboid(behind_left, behind_bottom, behind_front,
                               behind_right, behind_top, behind_back)
        
        existing = []
        for cub in (above_cuboid, below_cuboid, left_cuboid, right_cuboid, infront_cuboid, behind_cuboid):
            if cub.exists():
                existing.append(cub)
                
        return existing
        

In [90]:
test = Cuboid(0, 0, 0, 1, 1, 1)
test2 = Cuboid(0, 0, 0, 0.5, 0.5, 0.5)

assert test.is_intersect(test2)
assert test.exists()
assert test2.exists()
assert test2.is_intersect(test)

intersect = test.find_intersect(test2)
assert intersect.exists()

split_vol = sum(cub.get_volume() for cub in test.split_around_intersect(intersect))
assert split_vol + intersect.get_volume() == test.get_volume()

split_vol = sum(cub.get_volume() for cub in test2.split_around_intersect(intersect))
assert split_vol + intersect.get_volume() == test2.get_volume()

assert test.state == "on"

I now have a Cuboid class that I can use to model regions.

I basically want to end up with a big list of Cuboids that are all *on*, so I can sum up the volume of each Cuboid in the list to get the final answer.

To do this, I need to go through each reactor step. For each:

1. Create a Cuboid with the dimensions given. REMEMBER: -0.5 to the smaller coord, +0.5 to the bigger coord. This is because in the problem is not a continuous space, and is instead made of cells. E.g. a region given by "x=1..2" needs a width of *2*, not a width of *1* which is 2-1. The 0.5 addition / subtraction fixes this.
2. Compare the new Cuboid to every Cuboid currently in the list, and check if it intersects with them. If it does, find `cuboid_in_list.split_around_intersect(cuboid_in_list.find_intersect(new_cuboid))`, and replace the cuboid in the list with its splits. This is to avoid overcounting and to deal with replacing parts of a Cuboid with an "off" Cuboid.
3. If the new Cuboid is an "on" Cuboid, add it to the list.
4. Go to the next step

In [100]:
steps = []

# with open("ex1.txt") as f:
# with open("ex2.txt") as f:
# with open("ex3.txt") as f:
with open("reactor.txt") as f:
    for line in f.readlines():
        stripped = line.strip()
        command  = stripped.split(" ")[0]
        x_range = stripped.split("x=")[1].split(",")[0]
        y_range = stripped.split("y=")[1].split(",")[0]
        z_range = stripped.split("z=")[1]
        xs = tuple(int(i) for i in x_range.split(".."))
        ys = tuple(int(i) for i in y_range.split(".."))
        zs = tuple(int(i) for i in z_range.split(".."))
        steps.append(tuple((command, xs, ys, zs)))
        
# print(steps)

In [103]:
on_regions = []
for step in steps:
    new_cuboid = Cuboid(step[1][0]-0.5, step[2][0]-0.5, step[3][0]-0.5,
                        step[1][1]+0.5, step[2][1]+0.5, step[3][1]+0.5, state=step[0])
    to_add = []
    to_remove = []
    for cub in on_regions:
        if new_cuboid.is_intersect(cub):
            intersect = cub.find_intersect(new_cuboid)
            cub_splits = cub.split_around_intersect(intersect)
            to_remove.append(cub)
            to_add.extend(cub_splits)
            
    for cub in to_remove:
        on_regions.remove(cub)
        
    on_regions.extend(to_add)
            
    if new_cuboid.state == "on":
        on_regions.append(new_cuboid)
        
total_volume = sum(cuboid.get_volume() for cuboid in on_regions)
# for cub in on_regions:
#     print(cub)
    
print(int(total_volume))

1218645427221987
